import pandas as pd
import os

all_transactions = pd.DataFrame()
all_p2p = pd.DataFrame()

data_folder = r'C:\PROJECTS\VS CODE'

# Названия столбцов для файлов с транзакциями
transaction_column_names = [
    'id', 'name', 'account_type', 'client_status', 'city', 
    'timestamp', 'category', 'amount', 'currency'
]

# Названия столбцов для файлов с переводами
p2p_column_names = [
    'id', 'name', 'account_type', 'client_status', 'city', 
    'timestamp', 'transfer_type', 'transfer_direction', 'amount', 'currency'
]

for filename in os.listdir(data_folder):
    file_path = os.path.join(data_folder, filename)
    
    # Проверка, является ли файл транзакциями
    if 'transactions' in filename:
        try:
            client_df = pd.read_csv(file_path, skiprows=1, names=transaction_column_names, encoding='UTF-8')
        except UnicodeDecodeError:
            client_df = pd.read_csv(file_path, skiprows=1, names=transaction_column_names, encoding='latin-1')
        all_transactions = pd.concat([all_transactions, client_df], ignore_index=True)
        
    # Проверка, является ли файл переводами (p2p или card)
    elif 'transfers' in filename or 'card' in filename:
        try:
            client_df = pd.read_csv(file_path, skiprows=1, names=p2p_column_names, encoding='UTF-8')
        except UnicodeDecodeError:
            client_df = pd.read_csv(file_path, skiprows=1, names=p2p_column_names, encoding='UTF-8')
        all_p2p = pd.concat([all_p2p, client_df], ignore_index=True)

print("Транзакции:")
print(all_transactions)
print("\nПереводы:")
print(all_p2p)

In [240]:
import pandas as pd
import numpy as np

# Предполагаем, что у вас уже есть DataFrame `all_transactions` из предыдущих шагов
# Если нет, загрузите его снова

# 1. Сгруппируем данные по клиенту и категории, чтобы получить общие траты
category_spending = all_transactions.groupby(['id', 'name', 'category'])['amount'].sum().reset_index()

# 2. Создадим DataFrame для хранения расчётов выгоды
client_benefits = pd.DataFrame(columns=['id', 'name', 'recommended_product', 'potential_benefit'])

# 3. Переберём каждого клиента и рассчитаем выгоду
for client_id in category_spending['id'].unique():
    client_data = category_spending[category_spending['id'] == client_id]
    
    # Инициализируем максимальную выгоду и лучший продукт
    max_benefit = 0
    best_product = 'Накопительный депозит'
    
    # Рассчитаем выгоду для карты с кэшбэком
    cashback_categories = ['Кафе и рестораны', 'Супермаркеты', 'Развлечения']
    cashback_sum = client_data[client_data['category'].isin(cashback_categories)]['amount'].sum()
    cashback_benefit = cashback_sum * 0.05  # 5% кэшбэка
    
    if cashback_benefit > max_benefit:
        max_benefit = cashback_benefit
        best_product = 'Карта с кэшбэком'
    
    # Рассчитаем выгоду для карты с милями
    miles_categories = ['Путешествия', 'Авиабилеты']
    miles_sum = client_data[client_data['category'].isin(miles_categories)]['amount'].sum()
    miles_benefit = miles_sum * 10  # 10 миль за 1 KZT
    
    if miles_benefit > max_benefit:
        max_benefit = miles_benefit
        best_product = 'Карта с милями'
        
    # Рассчитаем выгоду для накопительного депозита
    # Для простоты, допустим, средний баланс клиента - 100,000 KZT
    deposit_benefit = 100000 * 0.10 / 12 * 3 # 10% годовых за 3 месяца
    
    if deposit_benefit > max_benefit:
        max_benefit = deposit_benefit
        best_product = 'Накопительный депозит'

    # Добавим результат в наш итоговый DataFrame
    client_info = client_data.iloc[0]
    client_benefits = pd.concat([client_benefits, pd.DataFrame([{
        'id': client_id,
        'name': client_info['name'],
        'recommended_product': best_product,
        'potential_benefit': max_benefit
    }])], ignore_index=True)

# 4. Выведем результат, отсортировав по выгоде
print(client_benefits.sort_values(by='potential_benefit', ascending=False))

for index, row in client_benefits.sort_values(by='potential_benefit', ascending=False).iterrows():
    message = f"Привет, {row['name']}! Мы проанализировали твои траты и нашли идеальный продукт для тебя: {row['recommended_product']}! Твоя потенциальная выгода может составить до {row['potential_benefit']:.2f} KZT. Узнай больше в приложении."
    print(message)
    print("---")

   id    name    recommended_product  potential_benefit
3  56  Нуркен         Карта с милями       1.466374e+06
4  57   Жания       Карта с кэшбэком       5.326745e+04
6  59  Динара       Карта с кэшбэком       4.743925e+04
7  60   Ермек       Карта с кэшбэком       2.745702e+04
0   7  Руслан       Карта с кэшбэком       2.248453e+04
5  58  Султан       Карта с кэшбэком       2.215028e+04
2  43  Нуртас  Накопительный депозит       2.500000e+03
1  42  Рустем  Накопительный депозит       2.500000e+03
Привет, Нуркен! Мы проанализировали твои траты и нашли идеальный продукт для тебя: Карта с милями! Твоя потенциальная выгода может составить до 1466374.00 KZT. Узнай больше в приложении.
---
Привет, Жания! Мы проанализировали твои траты и нашли идеальный продукт для тебя: Карта с кэшбэком! Твоя потенциальная выгода может составить до 53267.45 KZT. Узнай больше в приложении.
---
Привет, Динара! Мы проанализировали твои траты и нашли идеальный продукт для тебя: Карта с кэшбэком! Твоя потенциал

C:\Users\iwbhappy\AppData\Local\Temp\ipykernel_1356\2254347742.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  client_benefits = pd.concat([client_benefits, pd.DataFrame([{


In [241]:
# Преобразование столбца 'amount' в числовой формат
all_transactions['amount'] = pd.to_numeric(all_transactions['amount']) # type: ignore

# Преобразование столбца 'timestamp' в формат даты и времени
all_transactions['timestamp'] = pd.to_datetime(all_transactions['timestamp']) # type: ignore

# Проверка, что типы данных изменились
print(all_transactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             2398 non-null   int64         
 1   name           2398 non-null   object        
 2   account_type   2398 non-null   object        
 3   client_status  2398 non-null   object        
 4   city           2398 non-null   object        
 5   timestamp      2398 non-null   datetime64[ns]
 6   category       2398 non-null   object        
 7   amount         2398 non-null   float64       
 8   currency       2398 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 168.7+ KB
None


In [242]:
# Группируем по id клиента и категории и суммируем траты
category_spending = all_transactions.groupby(['id', 'category'])['amount'].sum().reset_index()

print(category_spending)

    id                category      amount
0    7                     АЗС   195545.89
1    7               Едим дома   170273.36
2    7             Играем дома   200163.54
3    7        Кафе и рестораны   449690.57
4    7                    Кино   150285.89
5    7  Косметика и Парфюмерия   422681.96
6    7        Продукты питания   767096.37
7    7            Смотрим дома   110980.10
8    7                   Такси   235104.21
9   42                   Такси  2254908.50
10  43                   Такси  1845105.08
11  56                     АЗС   161763.76
12  56               Едим дома   139067.38
13  56             Играем дома   179500.84
14  56        Кафе и рестораны   541118.92
15  56                    Кино   109659.84
16  56                   Отели   224207.33
17  56        Продукты питания   826214.65
18  56             Путешествия   146637.40
19  56             Развлечения    10104.26
20  56            Смотрим дома   189311.80
21  56                   Такси   211080.97
22  57     

In [243]:
# Создаем функцию для рекомендации продукта
def recommend_product(category):
    if category in ['Кафе и рестораны', 'Супермаркеты', 'Развлечения']:
        return 'Карта с кэшбэком'
    elif category in ['Путешествия', 'Авиабилеты']:
        return 'Карта с милями'
    else:
        return 'Накопительный депозит'

# Применяем функцию к нашим данным
category_spending['recommended_product'] = category_spending['category'].apply(recommend_product)

# Расчет потенциальной выгоды (например, 5% кэшбэка)
cashback_rate = 0.05
category_spending['potential_benefit'] = category_spending['amount'] * cashback_rate

print(category_spending)

    id                category      amount    recommended_product  \
0    7                     АЗС   195545.89  Накопительный депозит   
1    7               Едим дома   170273.36  Накопительный депозит   
2    7             Играем дома   200163.54  Накопительный депозит   
3    7        Кафе и рестораны   449690.57       Карта с кэшбэком   
4    7                    Кино   150285.89  Накопительный депозит   
5    7  Косметика и Парфюмерия   422681.96  Накопительный депозит   
6    7        Продукты питания   767096.37  Накопительный депозит   
7    7            Смотрим дома   110980.10  Накопительный депозит   
8    7                   Такси   235104.21  Накопительный депозит   
9   42                   Такси  2254908.50  Накопительный депозит   
10  43                   Такси  1845105.08  Накопительный депозит   
11  56                     АЗС   161763.76  Накопительный депозит   
12  56               Едим дома   139067.38  Накопительный депозит   
13  56             Играем дома   1

In [244]:
# Объединяем информацию о клиенте с расчётами
final_df = all_transactions[['id', 'name']].drop_duplicates().merge(category_spending, on='id') # type: ignore

# Сортируем по потенциальной выгоде, чтобы предлагать наиболее выгодные продукты
final_df = final_df.sort_values(by='potential_benefit', ascending=False)

# Формируем персонализированные сообщения для каждого клиента
for index, row in final_df.iterrows():
    message = f"Привет, {row['name']}! За последние 3 месяца ты потратил {row['amount']:.2f} KZT на '{row['category']}'. С нашей новой '{row['recommended_product']}' ты мог бы сэкономить до {row['potential_benefit']:.2f} KZT! ✨"
    print(message)
    print("---")

Привет, Рустем! За последние 3 месяца ты потратил 2254908.50 KZT на 'Такси'. С нашей новой 'Накопительный депозит' ты мог бы сэкономить до 112745.43 KZT! ✨
---
Привет, Нуртас! За последние 3 месяца ты потратил 1845105.08 KZT на 'Такси'. С нашей новой 'Накопительный депозит' ты мог бы сэкономить до 92255.25 KZT! ✨
---
Привет, Жания! За последние 3 месяца ты потратил 1065348.95 KZT на 'Кафе и рестораны'. С нашей новой 'Карта с кэшбэком' ты мог бы сэкономить до 53267.45 KZT! ✨
---
Привет, Динара! За последние 3 месяца ты потратил 1006297.39 KZT на 'Продукты питания'. С нашей новой 'Накопительный депозит' ты мог бы сэкономить до 50314.87 KZT! ✨
---
Привет, Динара! За последние 3 месяца ты потратил 948784.95 KZT на 'Кафе и рестораны'. С нашей новой 'Карта с кэшбэком' ты мог бы сэкономить до 47439.25 KZT! ✨
---
Привет, Султан! За последние 3 месяца ты потратил 928688.83 KZT на 'Продукты питания'. С нашей новой 'Накопительный депозит' ты мог бы сэкономить до 46434.44 KZT! ✨
---
Привет, Жания!

In [245]:
import numpy as np

# 1. Сгруппируем данные по клиенту и категории, чтобы получить общие траты
category_spending = all_transactions.groupby(['id', 'name', 'category'])['amount'].sum().reset_index()

# 2. Создадим пустой DataFrame для хранения результатов
final_report_df = pd.DataFrame(columns=['id', 'name', 'recommended_product', 'potential_benefit'])

# 3. Пройдём по каждому клиенту и рассчитаем выгоду
for client_id in category_spending['id'].unique():
    client_data = category_spending[category_spending['id'] == client_id]
    
    max_benefit = 0
    best_product = 'Накопительный депозит'
    
    # Расчёт выгоды для карты с кэшбэком
    cashback_categories = ['Кафе и рестораны', 'Супермаркеты', 'Развлечения']
    cashback_sum = client_data[client_data['category'].isin(cashback_categories)]['amount'].sum()
    cashback_benefit = cashback_sum * 0.05
    
    if cashback_benefit > max_benefit:
        max_benefit = cashback_benefit
        best_product = 'Карта с кэшбэком'
    
    # Расчёт выгоды для карты с милями
    miles_categories = ['Путешествия', 'Авиабилеты']
    miles_sum = client_data[client_data['category'].isin(miles_categories)]['amount'].sum()
    miles_benefit = miles_sum * 10
    
    if miles_benefit > max_benefit:
        max_benefit = miles_benefit
        best_product = 'Карта с милями'
        
    # Расчёт выгоды для накопительного депозита
    deposit_benefit = 100000 * 0.10 / 12 * 3 
    
    if deposit_benefit > max_benefit:
        max_benefit = deposit_benefit
        best_product = 'Накопительный депозит'

    # Добавляем строку с результатами в DataFrame
    client_info = client_data.iloc[0]
    new_row = pd.DataFrame([{
        'id': client_id,
        'name': client_info['name'],
        'recommended_product': best_product,
        'potential_benefit': max_benefit
    }])
    final_report_df = pd.concat([final_report_df, new_row], ignore_index=True)

# 4. Выводим итоговый DataFrame на экран
print(final_report_df)

   id    name    recommended_product  potential_benefit
0   7  Руслан       Карта с кэшбэком       2.248453e+04
1  42  Рустем  Накопительный депозит       2.500000e+03
2  43  Нуртас  Накопительный депозит       2.500000e+03
3  56  Нуркен         Карта с милями       1.466374e+06
4  57   Жания       Карта с кэшбэком       5.326745e+04
5  58  Султан       Карта с кэшбэком       2.215028e+04
6  59  Динара       Карта с кэшбэком       4.743925e+04
7  60   Ермек       Карта с кэшбэком       2.745702e+04


C:\Users\iwbhappy\AppData\Local\Temp\ipykernel_1356\1426240803.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_report_df = pd.concat([final_report_df, new_row], ignore_index=True)
